In [1]:
import mlflow
import pandas as pd
import mlflow.sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pandas as pd
import re
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import numpy as np

In [5]:
train_df=pd.read_csv(r"D:\projects\testing\notebooks\emotions\training.csv")
valid_df=pd.read_csv(r"D:\projects\testing\notebooks\emotions\validation.csv")
test_df=pd.read_csv(r"D:\projects\testing\notebooks\emotions\test.csv")

train_df.head()

,text,label
0,i didnt feel humiliated,0
1,i can go from feeling so hopeless to so damned...,0
2,im grabbing a minute to post i feel greedy wrong,3
3,i am ever feeling nostalgic about the fireplac...,2
4,i am feeling grouchy,3


sadness (0), joy (1), love (2), anger (3), fear (4)

In [6]:
train_df.isnull().sum()

text     0
label    0
dtype: int64

In [7]:
test_df.isnull().sum()

text     0
label    0
dtype: int64

Data Preprocessing

In [11]:
def lemmatization(text):
    """Lemmatizes the input text."""
    lemmatizer=WordNetLemmatizer()
    words=text.split()
    lemmatized_words=[]

    for word in words:
        word=lemmatizer.lemmatize(word)
        lemmatized_words.append(word)
    
    result_text=' '.join(lemmatized_words)
    return result_text

def lower_case(text):
    """Converts the input text to lower case."""
    return text.lower()

def remove_stop_words(text):
    """Removes stopwords from the input text."""
    stop_words=set(stopwords.words('english'))
    words=text.split()
    filtered_words=[]

    for word in words:
        if word not in stop_words:
            filtered_words.append(word)
    
    result_text=' '.join(filtered_words)
    return result_text

def removing_numbers(text):
    """Removes numbers from input_text."""
    filtered_words=[]
    words=text.split()

    for word in words:
        if not word.isdigit():
            filtered_words.append(word)
    result_text=' '.join(filtered_words)
    return result_text

def removing_urls(text):
    """Removes URLs from the text"""
    url_pattern = r'https?://\S+|www.\S+'
    result_text= re.sub(url_pattern, ' ', text)
    return result_text

def removing_punctuations(text):
    """Removes punctuation form text"""
    punctuation_pattern = f"[{re.escape(string.punctuation)}]"
    result_text = re.sub(punctuation_pattern, ' ', text)
    return result_text



In [15]:
def normalize_text(df):
    """Normalize the text data."""
    try:
        df['text'] = df['text'].apply(lower_case)
        df['text'] = df['text'].apply(remove_stop_words)
        df['text'] = df['text'].apply(removing_numbers)
        df['text'] = df['text'].apply(removing_punctuations)
        df['text'] = df['text'].apply(removing_urls)
        df['text'] = df['text'].apply(lemmatization)
        return df
    except Exception as e:
        print(f'Error during text normalization: {e}')
        raise

In [16]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\thaku\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\thaku\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\thaku\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [17]:
train_df= normalize_text(train_df)
valid_df= normalize_text(valid_df)

In [18]:
train_df.head()
valid_df.head()

,text,label
0,im feeling quite sad sorry ill snap soon,0
1,feel like still looking blank canvas blank pie...,0
2,feel like faithful servant,2
3,feeling cranky blue,3
4,treat feeling festive,1


In [21]:
train_df['label'].value_counts()

label
1    5362
0    4666
3    2159
4    1937
2    1304
5     572
Name: count, dtype: int64

In [22]:
test_df['label'].value_counts()

label
1    695
0    581
3    275
4    224
2    159
5     66
Name: count, dtype: int64

In [23]:
vectorizer = CountVectorizer(max_features=100)

X_train = vectorizer.fit_transform(train_df['text'])
y_train = train_df['label']

X_valid = vectorizer.transform(valid_df['text'])
y_valid = valid_df['label']

X_test = vectorizer.transform(test_df['text'])
y_test = test_df['label']

In [24]:
import dagshub

mlflow.set_tracking_uri('https://dagshub.com/harshitneverdebugs/testing.mlflow')
dagshub.init(repo_owner='harshitneverdebugs', repo_name='testing', mlflow=True)


mlflow.set_experiment("BiLSTM Baseline")

Accessing as harshitneverdebugs

Initialized MLflow to track repo "harshitneverdebugs/testing"

Repository harshitneverdebugs/testing initialized!

2025/05/26 01:27:00 INFO mlflow.tracking.fluent: Experiment with name 'BiLSTM Baseline' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/1dd70d45af314efdafee614ecfdf3010', creation_time=1748203020719, experiment_id='0', last_update_time=1748203020719, lifecycle_stage='active', name='BiLSTM Baseline', tags={}>